**сеть Хемминга**

нечёткий поиск по словарю

Евгений Борисов esborisov@sevsu.ru

---

_А.Арустамов, А.Стариков    Ассоциативная память.Применение сетей Хемминга для нечеткого поиска._
_https://basegroup.ru/community/articles/assoc_

In [1]:
import numpy as np
from numpy import random as rng
import gzip

##  загружаем данные 

In [2]:
def read_list_txt(fname):
    with gzip.open(fname,'rt') as f: 
        return [ w.strip() for w in f.read().split('\n') if w.strip() ]
        
ideal = read_list_txt('../data/towns/ideal_u.txt.gz')

max_word_len = max([len(w) for w in ideal]) # максимальная длинна слова словаре
voc_len = len(ideal) # размер словаря

voc_len, max_word_len

(112, 14)

In [3]:
ideal[:7]

['Акмола', 'Актау', 'Алматы', 'Архангельск', 'Астрахань', 'Ашхабад', 'Баку']

## кодируем данные

In [4]:
# Кодирование строится таким образом, 
# что бы стоящие рядом на компьютерной клавиатуре символы 
# имели близкие по Хеммингу коды.
# Таким образом должно достигаться наиболее эффективное исправление опечаток.
CODE_A = {
   'к':(0,0,0,0,0,),
   'ж':(0,0,0,0,1,),
   'г':(0,0,0,1,0,),
   'щ':(0,0,0,1,1,),
   'ч':(0,0,1,0,0,),
   'х':(0,0,1,0,1,),
   'ю':(0,0,1,1,0,),
   'с':(0,0,1,1,1,),
   'у':(0,1,0,0,0,),
   'б':(0,1,0,0,1,),
   'л':(0,1,0,1,0,),
   'е':(0,1,0,1,1,),
   'о':(0,1,1,0,0,),
   'р':(0,1,1,0,1,),
   'н':(0,1,1,1,0,),
   'й':(0,1,1,1,1,),
   'т':(1,0,0,0,0,),
   'з':(1,0,0,0,1,),
   'ы':(1,0,0,1,0,),
   'п':(1,0,0,1,1,),
   'ф':(1,0,1,0,0,),
   'ш':(1,0,1,0,1,),
   'м':(1,0,1,1,0,),
   'э':(1,0,1,1,1,),
   'ъ':(1,1,0,0,0,),
   'д':(1,1,0,0,1,),
   'в':(1,1,0,1,0,),
   'а':(1,1,0,1,1,),
   'ц':(1,1,1,0,0,),
   'и':(1,1,1,0,1,),
   'я':(1,1,1,1,0,),
   'ь':(1,1,1,1,1,)
}

CODE_V = { CODE_A[a]:a for a in CODE_A }

def encode(w): return  [ CODE_A[a] for a in list(w.lower()) ]
def decode(c): return  [ CODE_V[v] for v in c ]

In [5]:
code_len = len(CODE_A['а']) # длинна кода символа

In [6]:
# заменяем буквы на коды
def encode_text(text):
    return [ encode(w) for w in text]

# обрезаем длиные слова
def strip_text(text,max_word=0): 
    return [ w[:max_word_len] for w in text ] 

# дополняем коды коротких слов нулями
def pad_code(x,max_word=0): 
    code_len = len(x[0][0]) # длинна кода
    # максимальная длинна слова
    mwl = max([len(v) for v in x]) if max_word<1 else max_word
    # дополнение нулями
    z = [(0,)*code_len]*mwl
    # дополняем короткое слово
    return [ v + z[:(mwl-len(v))] for v in x ]

# собираем датасет из списка слов text,
# max_word - ограничение максимальной длины слова (0 - нет ограничений)
def make_dataset(text,max_word=0):
    t = strip_text(text,max_word) 
    x = encode_text(t)
    x = pad_code(x,max_word)
    return np.array([ sum(v,()) for v in x ])
    

In [7]:
x_train = make_dataset(ideal)   

# масштабируем в [-1,+1]
x_train = x_train*2-1

x_train.shape

(112, 70)

In [8]:
x_train

array([[ 1,  1, -1, ..., -1, -1, -1],
       [ 1,  1, -1, ..., -1, -1, -1],
       [ 1,  1, -1, ..., -1, -1, -1],
       ...,
       [ 1, -1,  1, ..., -1, -1, -1],
       [ 1,  1,  1, ..., -1, -1, -1],
       [ 1,  1,  1, ..., -1, -1, -1]])

## загружаем память сети

сеть Хэминга

![neural-net-hamming](http://mechanoid.su/content/neural-net-hamming-classifier.html/nnet.png)

In [9]:
# voc_len = len(ideal) # размер словаря
# size_layer_0 = code_len * (max_word_len*2) # размер входого слоя, (кодированное слово)
# size_layer_1 = voc_len # # размер выходого слоя (индикатор позиции в словаре)

In [10]:
m = x_train.shape[0] # количество учебных примеров 
c = 1.0/(2.0*m) # коэффициент торможения
W1 = (0.5*x_train).T # веса для нейронов первого слоя
W2 = -(np.ones(m)-np.eye(m))*c+np.eye(m) # веса для нейронов второго слоя

In [11]:
with np.printoptions(formatter={'float':'{: 0.2f}'.format}): print(W1) 

[[ 0.50  0.50  0.50 ...  0.50  0.50  0.50]
 [ 0.50  0.50  0.50 ... -0.50  0.50  0.50]
 [-0.50 -0.50 -0.50 ...  0.50  0.50  0.50]
 ...
 [-0.50 -0.50 -0.50 ... -0.50 -0.50 -0.50]
 [-0.50 -0.50 -0.50 ... -0.50 -0.50 -0.50]
 [-0.50 -0.50 -0.50 ... -0.50 -0.50 -0.50]]


In [12]:
# with np.printoptions(precision=4, suppress=True): print(W2)
with np.printoptions(formatter={'float':'{: 0.3f}'.format}): print(W2) 

[[ 1.000 -0.004 -0.004 ... -0.004 -0.004 -0.004]
 [-0.004  1.000 -0.004 ... -0.004 -0.004 -0.004]
 [-0.004 -0.004  1.000 ... -0.004 -0.004 -0.004]
 ...
 [-0.004 -0.004 -0.004 ...  1.000 -0.004 -0.004]
 [-0.004 -0.004 -0.004 ... -0.004  1.000 -0.004]
 [-0.004 -0.004 -0.004 ... -0.004 -0.004  1.000]]


In [13]:
def run(x,W1,W2,max_iter=500):
    o = x.dot(W1)
    for i in range(max_iter):
        o_prev=o.copy() # сохраняем состояние
        o = np.max( [ o.dot(W2), np.zeros(m) ], axis=0 ) # переходим в новое состояние
        # если состояние не изменилось то завершаем
        if np.all(o==o_prev): break
    #print(i)  
    return o        

## тестируем

In [14]:
test = read_list_txt('../data/towns/test_u.txt.gz')
len(test)

112

In [15]:
test[:7]

['Акмьлу', 'Актуу', 'Алмуты', 'Архунгельск', 'Аструхунь', 'Ашхуюуд', 'Буку']

In [16]:
max_word_len = max([len(w) for w in ideal])
x_test = make_dataset(test,max_word=max_word_len)    
x_test = x_test*2-1
x_test.shape

(112, 70)

In [17]:
from tabulate import tabulate 

out = [ np.argmax(run(x_test[i,:],W1,W2)) for i in range(x_test.shape[0]) ]
res = [ [t, ideal[out[i]] ] for i,t in enumerate(test) ]

tabulate(res, tablefmt='html')

Акмьлу,Актау
Актуу,Актау
Алмуты,Алматы
Архунгельск,Архангельск
Аструхунь,Астрахань
Ашхуюуд,Ашхабад
Буку,Баку
Бурнуул,Барнаул
Билкек,Бишкек
Блугьвещенск,Благовещенск
Брутск,Братск
